In [1]:
from pydub import AudioSegment
from pydub.utils import mediainfo
from glob import glob
import h5py
import numpy as np
import os

In [2]:
dirs = glob('../fma_small/*')
#dirs = dirs[:2]
dirs = [str(dirs[i]) + str('/*.mp3') for i in range(len(dirs))]
dirs = [glob(dirs[i]) for i in range(len(dirs))]
path = []
for i in range(len(dirs)):
    path += dirs[i]

In [4]:
#path = path[:5]
len(path)

5

In [4]:
file = h5py.File("whole.hdf5", "w")

In [5]:
Fs = 44100
INPUT_LEN = 30*Fs

for i in range(len(path)):
    label = int(path[i][17:23])
    music = AudioSegment.from_file(path[i])
    data = np.array(music.get_array_of_samples())
    
    # check if the audio has two channels
    if len(data) > INPUT_LEN: 
        data = [data[i] for i in range(0,len(data),2)] #get left channel only
            
    # save data
    data = data/np.amax(data)
    dset = file.create_dataset(str(label), data=data)
    #print('current song num: '+str(dset.name)+'num music: '+str(i))

In [70]:
Fs = 44100
LENGTH = int(1.5*Fs)
INPUT_LEN = 30*Fs
INTERVAL = int(0.05*Fs) #in seconds, determines how many fingerprints each song will generate

for i in range(len(path)):
    label = int(path[i][17:23])
    music = AudioSegment.from_file(path[i])
    temp = np.array(music.get_array_of_samples())
    
    # check if the audio has two channels
    if len(temp) > INPUT_LEN: 
        temp = [temp[i] for i in range(0,len(temp),2)] #get left channel only
            
    # convert to fingerprints
    cur_data = []
    num_fp = int((len(temp)-LENGTH)/INTERVAL)+1
    for j in range(num_fp):
        segment = temp[j*INTERVAL:j*INTERVAL+LENGTH]
        segment = np.asarray(segment)          
        cur_data.append(segment)
        
    cur_data = np.asarray(cur_data)
    cur_data = cur_data.astype(np.float32)
    cur_data = cur_data/np.amax(cur_data)
    dset = file.create_dataset(str(label), data=cur_data)
    print('current song num: '+str(dset.name)+'num music: '+str(i))
    

current song num: /2num music: 0
current song num: /5num music: 1
current song num: /10num music: 2
current song num: /140num music: 3
current song num: /141num music: 4
current song num: /148num music: 5
current song num: /182num music: 6
current song num: /190num music: 7
current song num: /193num music: 8
current song num: /194num music: 9
current song num: /197num music: 10
current song num: /200num music: 11
current song num: /203num music: 12
current song num: /204num music: 13
current song num: /207num music: 14
current song num: /210num music: 15
current song num: /211num music: 16
current song num: /212num music: 17
current song num: /213num music: 18
current song num: /255num music: 19
current song num: /256num music: 20
current song num: /368num music: 21
current song num: /424num music: 22
current song num: /459num music: 23
current song num: /534num music: 24
current song num: /540num music: 25
current song num: /546num music: 26
current song num: /574num music: 27
current

In [6]:
file.close()

In [9]:
f = h5py.File("whole.hdf5", "r")
for label in f:
    print(f[label])
    
f.close()

<HDF5 dataset "135043": shape (1323648,), type "<f8">
<HDF5 dataset "135054": shape (1323648,), type "<f8">
<HDF5 dataset "135091": shape (1322496,), type "<f8">
<HDF5 dataset "135336": shape (1322496,), type "<f8">
<HDF5 dataset "135337": shape (1323648,), type "<f8">


In [23]:
data = np.random.rand(10,10)
dset = file.create_dataset("init", data=data)


In [68]:
os.remove("test.hdf5")

In [24]:
f = h5py.File("test.hdf5", "r")
dset = f['init']

In [25]:
f = h5py.File("test.hdf5", "a")
grp = f.create_group('subgroup')

In [26]:
dset2 = grp.create_dataset('another', data=np.random.rand(5,5)*10)

In [28]:
dset3 = f.create_dataset('subgroup2/hahadataset', data=np.random.rand(2,2)*100)